In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Read the data
data = pd.read_csv(r'.\output\(37t)สถานีอุตุนิยมวิทยาลำปาง.csv')

# Drop non-numeric columns and handle missing values
data_clean = data.drop(columns=['datetime', 'date', 'hour'])
data_clean = data_clean.fillna(method='ffill').fillna(method='bfill')

# Extract SO2 values
so2_values = data_clean[' SO2 '].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data_clean)
scaled_so2 = scaler.fit_transform(so2_values)

# Create sequences for LSTM
def create_sequences(data, label, time_steps=1):
    x, y = [], []
    for i in range(len(data) - time_steps):
        x.append(data[i:(i + time_steps)])
        y.append(label[i + time_steps])
    return np.array(x), np.array(y)

TIME_STEPS = 24
X, y = create_sequences(scaled_data, scaled_so2, TIME_STEPS)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


FileNotFoundError: [Errno 2] No such file or directory: '.\\output\\(37t)สถานีอุตุนิยมวิทยาลำปาง.csv'

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np






# Define the LSTM model
model = Sequential()

model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.1,
    shuffle=False
)



In [ ]:
import matplotlib.pyplot as plt


# Plot training history
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

# Setting TensorFlow to use a GPU
tf.debugging.set_log_device_placement(False)
tf.get_logger().setLevel('ERROR')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Read and preprocess the data
data = pd.read_csv(r'D:\Users\User\PycharmProjects\may-project\output\(37t)สถานีอุตุนิยมวิทยาลำปาง.csv')
data_clean = data.drop(columns=['datetime', 'date', 'hour'])
data_clean = data_clean.fillna(method='ffill').fillna(method='bfill')
so2_values = data_clean[' SO2 '].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data_clean)
scaled_so2 = scaler.fit_transform(so2_values)

# Create sequences for LSTM
def create_sequences(data, label, time_steps=1):
    x, y = [], []
    for i in range(len(data) - time_steps):
        x.append(data[i:(i + time_steps)])
        y.append(label[i + time_steps])
    return np.array(x), np.array(y)

TIME_STEPS = 24
X, y = create_sequences(scaled_data, scaled_so2, TIME_STEPS)

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the entire dataset
model.fit(X, y, epochs=10, batch_size=2048)

# Predict the next 2 months (1440 hours)
input_sequence = X[-1:].copy()
predictions = []

for i in range(1440):
    new_pred = model.predict(input_sequence)
    predictions.append(new_pred[0])
    new_seq = np.append(input_sequence[0, 1:], new_pred, axis=0)
    input_sequence[0] = new_seq

# Convert predictions back to the original scale
predictions_original_scale = scaler.inverse_transform(predictions)

# Plot the predictions
plt.figure(figsize=(15, 6))
plt.plot(predictions_original_scale)
plt.title("SO2 Predictions for the Next 2 Months")
plt.xlabel("Hours")
plt.ylabel("SO2 Level")
plt.grid(True)
plt.show()


 5/30 [====>.........................] - ETA: 2s - loss: 0.0079Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_52152 in device /job:localhost/replica:0/task:0/device:GPU:0
 6/30 [=====>........................] - ETA: 2s - loss: 0.0075Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_52152 in device /job:localhost/replica:0/task:0/device:GPU:0
 7/30 [======>.......................] - ETA: 2s - loss: 0.0072Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_52152 in device /job:localhost/replica:0/task:0/device:GPU:0
 8/30 [=======>.............

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 16 and the array at index 1 has size 1

In [ ]:
# Predict the next 2 months (1440 hours)
# Predict the next 2 months (1440 hours)
input_sequence = X[-1:].copy()

predictions = []

for i in range(1440):
    new_pred = model.predict(input_sequence)
    predictions.append(new_pred[0])

    # Reshape new_pred for concatenation
    new_pred_reshaped = new_pred.reshape(1, -1)

    # Construct the new sequence
    new_seq = np.concatenate((input_sequence[0, 1:], new_pred_reshaped), axis=0)

    # Update input_sequence for the next prediction
    input_sequence = new_seq.reshape(1, TIME_STEPS, scaled_data.shape[1])

# Plot the predictions
plt.figure(figsize=(15, 6))
plt.plot(predictions_original_scale)
plt.title("SO2 Predictions for the Next 2 Months")
plt.xlabel("Hours")
plt.ylabel("SO2 Level")
plt.grid(True)
plt.show()


Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptionsDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptionsDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op 

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 16 and the array at index 1 has size 1

In [ ]:
import tensorflow as tf
tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
gpus


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1
